In [ ]:
import pandas as pd
import numpy as np
import os
import cv2 as cv
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, AveragePooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations, layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models

In [ ]:
data_dir = '../input/gtsrb-german-traffic-sign'
train_df = pd.read_csv(os.path.join(data_dir, 'Train.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'Test.csv'))
img = cv.imread(os.path.join(data_dir, train_df.iloc[0].Path))
img.shape

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df['ClassId'] = train_df['ClassId'].astype(str)
test_df['ClassId'] = test_df['ClassId'].astype(str)

In [ ]:
train_image_data = ImageDataGenerator(rescale = 1./255,
                                      shear_range = 0.2,
                                      zoom_range = 0.2)
val_image_data = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_image_data.flow_from_dataframe(dataframe = train_df,
                                                       directory = data_dir,
                                                       x_col = 'Path',
                                                       y_col = 'ClassId',
                                                       batch_size = 32,
                                                       class_mode = 'categorical',
                                                       target_size = (224, 224))
test_generator = val_image_data.flow_from_dataframe(dataframe = test_df,
                                                    directory = data_dir,
                                                    x_col = 'Path',
                                                    y_col = 'ClassId',
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

In [ ]:
vggmodel = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3),pooling='max')
vggmodel.trainable = False

In [ ]:
model = models.Sequential([vggmodel,
                           layers.Flatten(),
                           layers.Dense(256, activation = 'relu'),
                           layers.Dropout(0.5),
                           layers.Dense(43, activation='softmax')])

In [ ]:
from keras.applications import Xception
model = Sequential([
    Xception(include_top=False, weights=None, input_shape=(224, 224, 3), pooling='avg'),
    Dense(43, activation = "softmax")]
    )
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# model.compile(loss = 'categorical_crossentropy', optimizer='adam' ,metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator,
                    validation_data = test_generator, 
                    epochs = 5, 
                    verbose = 1)